In [1]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from transformers import ViTModel, ViTFeatureExtractor

In [9]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/1. Completed_Data/CanonBeetles.csv"
canon_df = pd.read_csv(file_path)

In [10]:
canon_df["ImageFilePath"] = canon_df["cropped_image_path"].apply(lambda x: f"/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/CANON/{x}")

In [11]:
cols = ['ImageFilePath', 'ScientificName']
df = canon_df[cols]
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax konanus
1,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax konanus


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")


In [13]:
def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = feature_extractor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        features = model(**inputs).last_hidden_state[:, 0, :]

    return features.cpu().numpy()
    

In [16]:
X = np.vstack([extract_features(img) for img in df["ImageFilePath"]])

print(f"Extracted feature shape: {X.shape}")


le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Extracted feature shape: (1622, 768)


In [17]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [28]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 32), alpha=1e-3, max_iter=300, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 63.69% | Prec: 73.75% | Rec: 63.69% | F1: 67.44% | Bal Acc: 35.17% | MCC: 0.5053
LogisticRegression        | Acc: 88.62% | Prec: 85.77% | Rec: 88.62% | F1: 86.24% | Bal Acc: 40.75% | MCC: 0.8233
SVMLinear                 | Acc: 90.77% | Prec: 89.41% | Rec: 90.77% | F1: 89.86% | Bal Acc: 57.80% | MCC: 0.8585
SVMPolynomial             | Acc: 84.62% | Prec: 83.52% | Rec: 84.62% | F1: 81.07% | Bal Acc: 31.64% | MCC: 0.7600
SVMRadialBasis            | Acc: 81.54% | Prec: 72.84% | Rec: 81.54% | F1: 75.46% | Bal Acc: 21.01% | MCC: 0.7106
NearestNeighbor           | Acc: 76.62% | Prec: 73.57% | Rec: 76.62% | F1: 73.88% | Bal Acc: 28.41% | MCC: 0.6292
RandomForest              | Acc: 76.62% | Prec: 65.66% | Rec: 76.62% | F1: 68.37% | Bal Acc: 17.75% | MCC: 0.6258
MLP_Baseline              | Acc: 90.15% | Prec: 89.56% | Rec: 90.15% | F1: 89.42% | Bal Acc: 51.70% | MCC: 0.8487


In [29]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
135,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax discedens,Mecyclothorax brunneonubiger,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax discedens
844,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax konanus,Mecyclothorax discedens,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus


In [30]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/17.ViT-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/17.ViT-linear-probing-species-metrics.csv", index=False)